In [1]:
#API DOCUMENTATION - https://api-docs.iqair.com/?version=latest#get-started

import requests
import pandas as pd
import json
import time

In [2]:
#Load German states
url = "http://api.airvisual.com/v2/states?country=Germany&key=df7d5666-30f3-4807-ae5f-d433698a4153"

payload={}
files={}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload, files=files)

#print(response.text.encode('utf8'))

states=response.text

#Convert states to a list of dictionaries
states = states[states.find("["):states.find("]")+1]
states_json = json.loads(states)
states_json

[{'state': 'Baden-Wuerttemberg'},
 {'state': 'Bavaria'},
 {'state': 'Berlin'},
 {'state': 'Brandenburg'},
 {'state': 'Bremen'},
 {'state': 'Hamburg'},
 {'state': 'Hessen'},
 {'state': 'Lower Saxony'},
 {'state': 'Mecklenburg-Vorpommern'},
 {'state': 'Nordrhein-Westfalen'},
 {'state': 'Rheinland-Pfalz'},
 {'state': 'Saarland'},
 {'state': 'Saxony'},
 {'state': 'Saxony-Anhalt'},
 {'state': 'Schleswig-Holstein'},
 {'state': 'Thuringia'}]

In [3]:
#Example for loop (just for training)
#for i in range(len(states_json)):
#    print(states_json[i]['state'])

Baden-Wuerttemberg
Bavaria
Berlin
Brandenburg
Bremen
Hamburg
Hessen
Lower Saxony
Mecklenburg-Vorpommern
Nordrhein-Westfalen
Rheinland-Pfalz
Saarland
Saxony
Saxony-Anhalt
Schleswig-Holstein
Thuringia


In [8]:
#Initialise data frame
data = []
payload={}
files={}
headers = {}

In [9]:
#Iterate over each state and each city & get data for each city

for i in range(len(states_json)-15):
    
    state = states_json[i]
    city_url = "http://api.airvisual.com/v2/cities?state={}&country=Germany&key=df7d5666-30f3-4807-ae5f-d433698a4153"
    new_city_url = city_url.format(state['state'])

    response = requests.request("GET", new_city_url, headers=headers, data=payload, files=files)
    
    #print(response.text.encode('utf8'))
    
    cities=response.text
    cities = cities[cities.find("["):cities.find("]")+1]
    cities_json = json.loads(cities)
    cities_json
    
    time.sleep(5)

    for j in range(len(cities_json)-22):

        city = cities_json[j]
        data_url = "http://api.airvisual.com/v2/city?city={}&state={}&country=Germany&key=df7d5666-30f3-4807-ae5f-d433698a4153"
        new_data_url = data_url.format(city['city'],state['state'])
                
        response_data = requests.request("GET", new_data_url, headers=headers, data=payload, files=files) 
        data_json = json.loads(response_data.text)
        print(data_json)
        time.sleep(10) #we can only get 5 requests per minute
        
        data.append([str(data_json['data']['city']), str(data_json['data']['state']),str(data_json['data']['country']),
                     str(data_json['data']['location']['coordinates']),
                     str(data_json['data']['current']['weather']['ts']),
                     str(data_json['data']['current']['weather']['tp']),
                     str(data_json['data']['current']['weather']['pr']),
                     str(data_json['data']['current']['weather']['hu']),
                     str(data_json['data']['current']['weather']['ws']),
                     str(data_json['data']['current']['weather']['wd']),
                     str(data_json['data']['current']['weather']['ic']),
                     str(data_json['data']['current']['pollution']['aqius']),
                     str(data_json['data']['current']['pollution']['mainus']),
                     str(data_json['data']['current']['pollution']['aqicn']),
                     str(data_json['data']['current']['pollution']['maincn']),
                    ])

{'status': 'success', 'data': {'city': 'Aalen', 'state': 'Baden-Wuerttemberg', 'country': 'Germany', 'location': {'type': 'Point', 'coordinates': [10.131, 48.812]}, 'current': {'weather': {'ts': '2021-03-18T18:00:00.000Z', 'tp': 0, 'pr': 1017, 'hu': 81, 'ws': 1.96, 'wd': 296, 'ic': '04n'}, 'pollution': {'ts': '2021-03-18T18:00:00.000Z', 'aqius': 41, 'mainus': 'p2', 'aqicn': 14, 'maincn': 'p2'}}}}
{'status': 'success', 'data': {'city': 'Baden-baden', 'state': 'Baden-Wuerttemberg', 'country': 'Germany', 'location': {'type': 'Point', 'coordinates': [8.243008, 48.758339]}, 'current': {'weather': {'ts': '2021-03-18T18:00:00.000Z', 'tp': 3, 'pr': 1020, 'hu': 65, 'ws': 2.57, 'wd': 200, 'ic': '01n'}, 'pollution': {'ts': '2021-03-18T18:00:00.000Z', 'aqius': 25, 'mainus': 'p2', 'aqicn': 9, 'maincn': 'p2'}}}}
{'status': 'success', 'data': {'city': 'Bernhausen', 'state': 'Baden-Wuerttemberg', 'country': 'Germany', 'location': {'type': 'Point', 'coordinates': [9.2249, 48.6763]}, 'current': {'weathe

In [10]:
df = pd.DataFrame(data, columns=["city", "state","country", "coordinates","timestamp", "temperature",
                                "percipitation", "humidity", "ws", "wd", "ic",
                                "aqius", "mainus", "aqicn", "maincn"])
df

,city,state,country,coordinates,timestamp,temperature,percipitation,humidity,ws,wd,ic,aqius,mainus,aqicn,maincn
0,Aalen,Baden-Wuerttemberg,Germany,"[10.131, 48.812]",2021-03-18T18:00:00.000Z,0,1017,81,1.96,296,04n,41,p2,14,p2
1,Baden-baden,Baden-Wuerttemberg,Germany,"[8.243008, 48.758339]",2021-03-18T18:00:00.000Z,3,1020,65,2.57,200,01n,25,p2,9,p2
2,Bernhausen,Baden-Wuerttemberg,Germany,"[9.2249, 48.6763]",2021-03-18T18:00:00.000Z,1,1018,65,1.54,170,02n,25,p2,9,p2
3,Biberach,Baden-Wuerttemberg,Germany,"[9.7902, 48.0951]",2021-03-18T18:00:00.000Z,1,1018,69,0.51,120,04n,37,p2,13,p2
4,Eggenstein,Baden-Wuerttemberg,Germany,"[8.39879, 49.09006]",2021-03-18T18:00:00.000Z,4,1015,57,1.86,261,03n,25,p2,18,o3
5,Eppelheim,Baden-Wuerttemberg,Germany,"[8.63644, 49.4019]",2021-03-18T18:00:00.000Z,4,1019,70,1.03,0,10n,71,p2,31,p2
6,Ettlingen,Baden-Wuerttemberg,Germany,"[8.40847, 48.967385]",2021-03-18T18:00:00.000Z,5,1015,57,1.76,237,03n,30,p2,10,p2
7,Freiburg,Baden-Wuerttemberg,Germany,"[7.809, 47.983]",2021-03-18T18:00:00.000Z,1,1018,82,1.6,51,04n,23,p2,20,n2
8,Friedrichshafen,Baden-Wuerttemberg,Germany,"[9.48, 47.6618]",2021-03-18T18:00:00.000Z,0,1019,75,1.54,100,02n,45,p2,17,p1
9,Gartringen,Baden-Wuerttemberg,Germany,"[8.906, 48.646]",2021-03-18T18:00:00.000Z,1,1018,65,1.54,170,02n,29,p2,10,p2


In [ ]:
######What we need to do

###SETUP WORK

#Create a storage & SQL database in Azure (Oli -next session)

###REQUEST HANDLING WORK - (Oli & Pily)

#Request the list of German cities (JN)
#if we cannot take all cities, lets exclude the smallest ones (max 300 cities)
#Request air quality & weather data for each city
#Make a df out of it:
#date (everyday)
#city
#country - Germany
#geolocation (might not be important to get from the api)
#all features

###MIGRATE DATA FROM PC TO AZURE
#dump data to the storage (Oli)
#dump data to SQL database (Oli)


###THINK ABOUT VIS

###Make the requests daily - append new data to old df & dump to Azure



In [ ]:
#Archived tasks:

#Set up Github repostiory (Pily)
#Push this file to Jupyter Notebook (Oli)
#How to workshop: requirements.txt & create virtual environment (Oli - next session)